## Import dependencies

Make sure you are in the wind_gust conda environment

Basemap is deprecated. Works in this environment because python was downgraded to 3.8.10. Cartopy provides similar features may be used in the future. 

In [ ]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.inspection import plot_partial_dependence

## Sample data

More data will be added here in the future, spanning different seasons. 

In [ ]:
Met_Train = Dataset(  # Metar Training
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/20170701_0000')
Met_Test = Dataset(  # Metar Testing
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/20180613_0000')
Hrrr = Dataset(  # HRRR
    '/Users/jesseturner/Documents/Wind_Gust_Prediction/Gust_data/extract2Da.esrl_hrrr.201718201001300.nc')

Met_Train.variables['windGust']
#Hrrr.variables['GUST_P0_L1_GLC0']

# Format metar data

In [ ]:
met_gust_train = Met_Train.variables['windGust']
met_lon_train = Met_Train.variables['longitude']
met_lat_train = Met_Train.variables['latitude']
met_temp_train = Met_Train.variables['temperature']
met_windspeed_train = Met_Train.variables['windSpeed']
met_winddir_train = Met_Train.variables['windDir']
met_press_train = Met_Train.variables['seaLevelPress']

met_gust_test = Met_Test.variables['windGust']
met_lon_test = Met_Test.variables['longitude']
met_lat_test = Met_Test.variables['latitude']
met_temp_test = Met_Test.variables['temperature']
met_windspeed_test = Met_Test.variables['windSpeed']
met_winddir_test = Met_Test.variables['windDir']
met_press_test = Met_Test.variables['seaLevelPress']

### Crop metar data to the continental US

There is likely a more efficient way to do this. The warning of "converting a masked element to nan" is currently expected, but may be changed to do manually in the future. 

In [ ]:
top = 49.3457868  # north lat
left = -124.7844079  # west lon
right = -66.9513812  # east lon
bottom = 24.7433195  # south lat

cell_train = []
for g, lon, lat, t, ws, wd, p in zip(met_gust_train, met_lon_train, met_lat_train, 
                                     met_temp_train, met_windspeed_train, met_winddir_train, met_press_train):
    if left <= lon <= right and bottom <= lat <= top:
        cell_train.append((g, lon, lat, t, ws, wd, p))
met_pos_train = np.asarray(cell_train)

cell_test = []
for g, lon, lat, t, ws, wd, p in zip(met_gust_test, met_lon_test, met_lat_test, 
                                     met_temp_test, met_windspeed_test, met_winddir_test, met_press_test):
    if left <= lon <= right and bottom <= lat <= top:
        cell_test.append((g, lon, lat, t, ws, wd, p))
met_pos_test = np.asarray(cell_test)

### Convert metar data to dataframe

In [ ]:
met_gust_train = met_pos_train[:, 0]
met_lon_train = met_pos_train[:, 1]
met_lat_train = met_pos_train[:, 2]
met_temp_train = met_pos_train[:, 3]
met_windspeed_train = met_pos_train[:, 4]
met_winddir_train = met_pos_train[:, 5]
met_press_train = met_pos_train[:, 6]


met_df_train = pd.DataFrame(
    {'MetWindGust': met_gust_train,
     'Longitude': met_lon_train,
     'Latitude': met_lat_train,
     'Temperature': met_temp_train,
     'Wind Speed': met_windspeed_train,
     'Wind Direction': met_winddir_train,
     'Pressure': met_press_train})

met_df_train = met_df_train.round(4)
met_df_train = met_df_train.replace(np.NaN, 0)
met_df_train

In [ ]:
met_gust_test = met_pos_test[:, 0]
met_lon_test = met_pos_test[:, 1]
met_lat_test = met_pos_test[:, 2]
met_temp_test = met_pos_test[:, 3]
met_windspeed_test = met_pos_test[:, 4]
met_winddir_test = met_pos_test[:, 5]
met_press_test = met_pos_test[:, 6]


met_df_test = pd.DataFrame(
    {'MetWindGust': met_gust_test,
     'Longitude': met_lon_test,
     'Latitude': met_lat_test,
     'Temperature': met_temp_test,
     'Wind Speed': met_windspeed_test,
     'Wind Direction': met_winddir_test,
     'Pressure': met_press_test})

met_df_test = met_df_test.round(4)
met_df_test = met_df_test.replace(np.NaN, 0)

### Plot metar locations

Uses the converted zeroes from the dataframe. 

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(met_df_train.Longitude, met_df_train.Latitude, latlon=True, c=met_df_train.MetWindGust,
          cmap='Purples', vmin=0, vmax=10, s=5)

plt.colorbar(label='Measured gusts (meter/sec)', extend='max')

plt.show()

### Training and running neural net

Currently, predictions are made with the data of the same time. Inputs are scaled so they have comparable impact. Standard Scaler: z = (x - u)/s. 

u = mean, s = standard deviation.

Two hidden layers: 100, 50. Used some rudimentary judging to come to this, testing and evaluating with RMSE. Read this for more info on the optimal amount: https://www.yourdatateacher.com/2021/05/10/how-many-neurons-for-a-neural-network/

Activation function: Currently using rectified linear activation function (relu), has a flat-then linear shape, will allow for more dynamic range. Will output the input directly if it is positive, otherwise, it will output zero.

Solver: adam. Uses back-propagration. 

Max Iterations: 1500. May be a little much. 

In [ ]:
array_train = met_df_train.values
X_train = array_train[:, 3:8]  # doesn't include lon, lat
y_train = array_train[:, 0]
scaler = StandardScaler()
rescaled_X_train = (scaler.fit_transform(X_train))

mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001,
                   power_t=0.5, max_iter=1500, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp = mlp.fit(rescaled_X_train, y_train)

array_test = met_df_test.values
X_test = array_test[:, 3:8]  # doesn't include lon, lat
y_test = array_test[:, 0]
rescaled_X_test = (scaler.fit_transform(X_test))

y_predict_mlp = mlp.predict(rescaled_X_test)

### Plot neural net predictions

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(array_test[:, 1], array_test[:, 2], latlon=True, c=y_predict_mlp, cmap='Purples',
          vmin=0, vmax=10, s=5)

plt.colorbar(label='predicted gusts (meter/sec)', extend='max')

plt.show()

### Evaluating neural net predictions

Root Mean Square Error, along with other interesting features of the predictions. 

Future: Add AUROC (Area Under the Curve of the Receiver Operating Characteristic)

In [ ]:
print('RMSE:', np.sqrt(
    metrics.mean_squared_error(y_test, y_predict_mlp)).round(4))
print('Mean Absolute Error:', 
    metrics.mean_absolute_error(y_test, y_predict_mlp).round(4))
print('R-Squared Score:', 
    metrics.r2_score(y_test, y_predict_mlp).round(4))

print('Range of Training: ', np.min(y_train), ' to ', np.max(y_train))
print('Range of Actual: ', np.min(y_test), ' to ', np.max(y_test))
print('Range of Predictions: ', np.min(y_predict_mlp).round(4), ' to ', np.max(y_predict_mlp).round(4))

### Loss Curve

Loss value evaluated at the end of each training step.

In [ ]:
plt.plot(mlp.loss_curve_)
plt.title("Loss Curve", fontsize=14)
plt.xlabel('Iterations')
plt.ylabel('Cost')
plt.show()
print('Loss:', mlp.loss_.round(4))
print('Best Loss:', mlp.best_loss_.round(4))
print('# of Training Samples:', mlp.t_)
print('# of Iterations:', mlp.n_iter_)
print('# of Layers:', mlp.n_layers_)
print('# of Outputs:', mlp.n_outputs_)
print('Output Activation Function:', mlp.out_activation_)

### Coefficients
The ith element in the list represents the weight matrix corresponding to layer i (scikit-learn docs).
Plots the weights for each neuron of each layer.

In [ ]:
plt.plot(mlp.coefs_[0])
plt.title("Coefficients", fontsize=14)
plt.xlabel('Neuron')
plt.ylabel('Weights')
plt.show()

### Intercepts
The ith element in the list represents the bias vector corresponding to layer i + 1 (scikit-learn docs).  

In [ ]:
plt.plot(mlp.intercepts_[0]) #Also 1, 2
plt.title("Intercepts", fontsize=14)
plt.xlabel('Neuron')
plt.ylabel('Bias')
plt.show()

### Histograms

Comparing the frequencies of wind-gust speeds between the METAR and the neural net.

In [ ]:
hist_metar = plt.hist(y_test, bins=30, color='navy', range=[0,30], )
plt.title("METAR Readings")
plt.ylim(0,200)

In [ ]:
hist_nn = plt.hist(y_predict_mlp, bins=30, color='orange', range=[0,30])
plt.title("Neural Net Predictions")
plt.ylim(0,200)

In [ ]:
plt.scatter(array_test[:, 1:2], y_test,  color='navy', s=5, alpha=1)
plt.scatter(array_test[:, 1:2], y_predict_mlp, color='orange', s=5, alpha=0.3)
plt.title("Scatter Comparison")
plt.xlabel("Longitude", size = 10)
plt.ylabel("Latitude", size = 10)

### Partial dependence plots
I don't understand these. Possibly need different features. 

In [ ]:
features = [0, 1, 2, 3]

display = plot_partial_dependence(
      mlp, rescaled_X_train, features, kind="both", subsample=50,
      n_jobs=3, grid_resolution=20, random_state=0
)

## Training neural net with only measured wind gusts
Wind gusts of zero are not included in training. 

Uses the same variables as the above, so make sure to clear before re-running any lines outside of this section.

In [ ]:
subset_met_df_train = met_df_train[met_df_train["MetWindGust"]!=0]
subset_met_df_train

In [ ]:
array_train = subset_met_df_train.values
X_train = array_train[:, 3:8]  # doesn't include lon, lat
y_train = array_train[:, 0]
scaler = StandardScaler()
rescaled_X_train = (scaler.fit_transform(X_train))

mlp = MLPRegressor(hidden_layer_sizes=(100, 50), activation='relu', solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='adaptive', learning_rate_init=0.001,
                   power_t=0.5, max_iter=1500, shuffle=True, random_state=None, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
mlp = mlp.fit(rescaled_X_train, y_train)

array_test = met_df_test.values
X_test = array_test[:, 3:8]  # doesn't include lon, lat
y_test = array_test[:, 0]
rescaled_X_test = (scaler.fit_transform(X_test))

y_predict_mlp = mlp.predict(rescaled_X_test)

In [ ]:
fig = plt.figure(figsize=(10, 10))
m = Basemap(projection='lcc', resolution='c',
            width=6E6, height=6E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
meridians = np.arange(10., 351., 20.)

m.scatter(array_test[:, 1], array_test[:, 2], latlon=True, c=y_predict_mlp, cmap='Purples',
          vmin=0, vmax=10, s=5)

plt.colorbar(label='predicted gusts (meter/sec)', extend='max')

plt.show()

In [ ]:
hist_nn = plt.hist(y_predict_mlp, bins=30, color='orange', range=[0,30])
plt.title("Neural Net Predictions")
plt.ylim(0,1500)

In [ ]:
plt.scatter(array_test[:, 1:2], y_test,  color='navy', s=5, alpha=1)
plt.scatter(array_test[:, 1:2], y_predict_mlp, color='orange', s=5, alpha=0.3)
plt.title("Scatter Comparison")
plt.xlabel("Longitude", size = 10)
plt.ylabel("Latitude", size = 10)

# Format HRRR data

In [ ]:
hrrr_lon = Hrrr.variables['gridlon_0']
hrrr_lat = Hrrr.variables['gridlat_0']
hrrr_gust = Hrrr.variables['GUST_P0_L1_GLC0']

hrrr_lon = np.reshape(hrrr_lon, 1905141)
hrrr_lat = np.reshape(hrrr_lat, 1905141)
hrrr_gust = np.reshape(hrrr_gust, 1905141)

hrrr_lon = hrrr_lon[0:1903585:1009]
hrrr_lat = hrrr_lat[0:1903585:1009]
hrrr_gust = hrrr_gust[0:1903585:1009]

### HRRR Predictions

In [ ]:
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='c',
            width=8E6, height=8E6,
            lat_0=45, lon_0=-100, )
m.drawcoastlines()
m.drawmapboundary()
m.drawcountries()
parallels = np.arange(0., 81, 10.)
m.drawparallels(parallels, labels=[False, True, True, False])
meridians = np.arange(10., 351., 20.)
m.drawmeridians(meridians, labels=[True, False, False, True])

m.scatter(hrrr_lon, hrrr_lat, latlon=True, c=hrrr_gust,
          cmap='coolwarm', vmin=5, vmax=10, s=9)
plt.show()